# Data

## Crowd human

In [ ]:
from pathlib import Path

crowdhuman_data_dir = Path("/home/jasseur/Downloads/crowdhuman")

In [ ]:
GENDERS = ["M", "F"]
AGES = ["1-2", "3-9", "10-20", "21-25", "26-27", "28-31", "32-36", "37-45", "46-54", "55-65", "66-116"]

In [ ]:
import pandas as pd
import numpy as np

def read_df(data_file):
    data_df = pd.read_csv(data_file)
    data_df.loc[pd.notnull(data_df["gender"]), "gender"] = (
        data_df.loc[pd.notnull(data_df["gender"]), "gender"].apply(lambda x: GENDERS.index(x))
    )
    data_df.loc[pd.notnull(data_df["age"]), "age"] = (
        data_df.loc[pd.notnull(data_df["age"]), "age"].apply(lambda x: AGES.index(x))
    )
    # data_df = data_df.groupby("id").filter(lambda d: d["gender"].notnull().any())
    return data_df

In [ ]:
crowdhuman_train_file = Path("../../../data/crowdhuman/annotation_train_with_classes.csv")
crowdhuman_train_df = read_df(crowdhuman_train_file)
crowdhuman_train_df.head(1)

In [ ]:
crowdhuman_train_df.shape

In [ ]:
crowdhuman_val_file = Path("../../../data/crowdhuman/annotation_val_with_classes.csv")
crowdhuman_val_df = read_df(crowdhuman_val_file)
crowdhuman_val_df.head(1)

In [ ]:
import random
import albumentations as A
import albumentations.augmentations.geometric.functional as F
import cv2

class AspectRatioResize(A.DualTransform):
    """Resize the input to the given height and width.

    Args:
        p (float): probability of applying the transform. Default: 1.
        height (int): desired height of the output.
        width (int): desired width of the output.
        interpolation (OpenCV flag): flag that is used to specify the interpolation algorithm. Should be one of:
            cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4.
            Default: cv2.INTER_LINEAR.

    Targets:
        image, mask, bboxes

    Image types:
        uint8, float32
    """

    def __init__(
        self, height, width, scale=(1, 1), interpolation=cv2.INTER_LINEAR, always_apply=False, p=1
    ):
        super(AspectRatioResize, self).__init__(always_apply, p)
        self.height = height
        self.width = width
        self.scale = scale
        self.interpolation = interpolation

    def apply(self, img, interpolation=cv2.INTER_LINEAR, **params):
        height, width, _ = img.shape
        r = min(self.width / width, self.height / height)
        scale = random.uniform(*self.scale)
        return F.resize(
            img, height=int(scale * r * height), width=int(scale * r * width),
            interpolation=interpolation
        )

    def apply_to_bbox(self, bbox, **params):
        # Bounding box coordinates are scale invariant
        return bbox

In [ ]:
import albumentations as A

IMAGE_SIZE = (1088, 608)

crowdhuman_train_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], scale=(1.0, 1.0), p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT, #BORDER_WRAP,
            value=[128, 128, 128]
        ),
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
        A.HorizontalFlip(p=0.5),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    ], 
    p=1.0, 
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["genders", "ages"]
    )
)

crowdhuman_val_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], scale=(1.0, 1.0), p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        )
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["genders", "ages"]
    )
)

In [ ]:
from src.train_dataset import TrainDataset
%load_ext autoreload
%autoreload 2

crowdhuman_train_dir = Path(crowdhuman_data_dir, "Images")
crowdhuman_train_dataset = TrainDataset(crowdhuman_train_dir, crowdhuman_train_df, crowdhuman_train_transform)
crowdhuman_val_dataset = TrainDataset(crowdhuman_train_dir, crowdhuman_val_df, crowdhuman_val_transform)

In [ ]:
len(crowdhuman_train_dataset), len(crowdhuman_val_dataset)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import cv2

image, centers, dimensions, genders, ages = crowdhuman_train_dataset[3].values()
for center, dimension, gender, age in zip(centers, dimensions, genders, ages):
    x, y = map(int, center)
    l, t, r, b = map(int, dimension)
    cv2.rectangle(
        image,
        (x - l, y - t),
        (x + r, y + b),
        (1.0, 0.0, 0.0) if gender == 1 else (0.0, 0.0, 1.0) if gender == 0 else (1.0, 1.0, 0.0),
        2
    )
    # if not np.isnan(age):
    #     cv2.putText(
    #         image, AGES[int(age)], (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (1, 1, 1), 2
    #     )

plt.figure()
plt.imshow(image)
plt.show()

## MOT15

In [ ]:
from pathlib import Path

mot15_data_dir = Path("/home/jasseur/Downloads/MOT15")
!ls $mot15_data_dir

In [ ]:
!ls /home/jasseur/Downloads/MOT15/train

In [ ]:
import pandas as pd

mot15_train_df = pd.read_csv(Path("../../../data/mot15/annotation_train.csv"))
mot15_train_df.head()

In [ ]:
import albumentations as A

IMAGE_SIZE = (1088, 608)

mot15_train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
    ], 
    p=1.0, 
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["person_ids"]
    )
)

mot15_val_transform = A.Compose(
    [
        A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["person_ids"]
    )
)

In [ ]:
import sys
sys.path.append("../..")
from person_reidentification.centernet.src.train_dataset import TrainDataset
%load_ext autoreload
%autoreload 2

mot15_train_dir = Path(mot15_data_dir, "train")
mot15_train_dataset = TrainDataset(mot15_train_dir, mot15_train_df, mot15_train_transform)

In [ ]:
def get_color(idx):
    idx = idx * 3
    color = ((37 * idx) % 255 / 255, (17 * idx) % 255 / 255, (29 * idx) % 255 / 255)
    return color

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import cv2

image, centers, person_ids = mot15_train_dataset[25].values()
for center, person_id in zip(centers.tolist(), person_ids.tolist()):
    x, y = map(int, center)
    color = get_color(person_id)
    cv2.circle(
        image, (x, y), 20, color, 2, lineType=cv2.LINE_AA
    )

plt.imshow(image)
plt.show()

# Modeling

## Training

In [ ]:
import os
import torch

def seedEverything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
!mkdir -p checkpoint log

In [ ]:
import sys
sys.path.append("../..")
from person_detection.centernet.src.pose_dla_dcn import get_pose_net as get_dla_dcn
from src.trainer import Trainer
%load_ext autoreload
%autoreload 2

num_layers = 34
heads = {"hm": 1, "wh": 4, "reg": 2, "id": 128, "gender": len(GENDERS), "age": len(AGES)}
head_conv = 256
net = get_dla_dcn(num_layers, heads, head_conv)
# model_path = Path("/home/jasseur/Downloads/crowdhuman_dla34.pth)"
model_path = Path("/home/jasseur/Downloads/fairmot_dla34.pth")
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda:1",
    checkpoint_dir="checkpoint", log_dir="log", model_path=None
)

In [ ]:
from torch.utils.data import ConcatDataset
train_dataset = ConcatDataset([crowdhuman_train_dataset, mot15_train_dataset])
val_dataset = ConcatDataset([crowdhuman_val_dataset, mot15_train_dataset])

In [ ]:
seedEverything(2020)
trainer.fit(train_dataset, val_dataset, batch_size=16, epochs=20)

In [ ]:
trainer.eval(val_dataset)